# 01 Server Rackmount Forecast

Quick validation notebook for the `data/processed/synosales_cleaned.parquet` output. Once the structure is confirmed, lift the aggregation logic into a production module under `src/` as part of the export-to-module plan.

In [1]:
import numpy as np
import os
import sys
from pathlib import Path
import pandas as pd
import warnings
import sklearn
import itertools
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression

In [2]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt



pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
repo = Path("/Volumes/docker/syno_bi/repo")
df = pd.read_parquet(repo / "data/processed/synosales_cleaned.parquet")

suite_NAS = df[df["source_sheet"].isin(["2023", "2024"])]
suite_C2 = df[df["source_sheet"].isin(["2023-C2", "2024-C2"])]

In [4]:
suite_NAS_subset = suite_NAS[["PI", "Customer", "ItemCode", "Product", "Quantity", "usd_adjusted_price", "usd_adjusted_total", "InvDate", "Country", "Type", "sub_cat", "Year", "Region", "source_sheet"]]
suite_C2_subset = suite_C2[["PI", "Customer", "ItemCode", "Product", "Quantity", "usd_adjusted_price", "usd_adjusted_total", "InvDate", "Country", "Type", "sub_cat", "Year", "Region", "source_sheet"]]

In [5]:
suite_NAS_subset["InvDate"] = pd.to_datetime(suite_NAS_subset["InvDate"])
suite_NAS_subset["usd_adjusted_total"] = suite_NAS_subset["usd_adjusted_total"].astype(float)

In [6]:
suite_NAS_clean = suite_NAS_subset[suite_NAS_subset["sub_cat"].fillna("").str.startswith("SVR-RM")].copy()

In [7]:
suite_NAS_clean.head(20)

,PI,Customer,ItemCode,Product,Quantity,usd_adjusted_price,usd_adjusted_total,InvDate,Country,Type,sub_cat,Year,Region,source_sheet
30911,268437.0,HORNINGTON,90-R368S0231,RS3618xs,2,2680.000,5306.400,2023-01-03,Hong Kong,SERVER,SVR-RM-GP-T2,2023,HQ,2023
30920,268753.0,HORNINGTON,90-R1221P231,RS1221+,4,979.000,3876.840,2023-01-03,Hong Kong,SERVER,SVR-RM-GP-T1,2023,HQ,2023
30921,268753.0,HORNINGTON,90-R1221R231,RS1221RP+,2,1335.000,2643.300,2023-01-03,Hong Kong,SERVER,SVR-RM-GP-T1,2023,HQ,2023
30922,268753.0,HORNINGTON,90-R422P0231,RS422+,1,679.000,672.210,2023-01-03,Hong Kong,SERVER,SVR-RM-GP-T1,2023,HQ,2023
30923,268753.0,HORNINGTON,90-RXC7R0231,RX1217RP,1,1600.000,1584.000,2023-01-03,Hong Kong,SERVER,SVR-RM-GX-T3,2023,HQ,2023
30927,261629.0,NAS2U,90-R368S0231,RS3618xs,12,2006.000,24072.000,2023-01-03,Malaysia,SERVER,SVR-RM-GP-T2,2023,HQ,2023
30931,267686.0,Mushor,90-R822P0231,RS822+,2,834.000,1668.000,2023-01-03,Saudi Arabia,SERVER,SVR-RM-GP-T1,2023,HQ,2023
30933,267687.0,Mushor,90-R36LX1231,RS3621xs+,1,3842.000,3842.000,2023-01-03,Saudi Arabia,SERVER,SVR-RM-GP-T3,2023,HQ,2023
30935,268279.0,Mushor,90-R822R0231,RS822RP+,1,1053.000,1053.000,2023-01-03,Saudi Arabia,SERVER,SVR-RM-GP-T1,2023,HQ,2023
30936,268279.0,Mushor,90-R169X0231,RS1619xs+,1,1542.000,1542.000,2023-01-03,Saudi Arabia,SERVER,SVR-RM-GP-T3,2023,HQ,2023


In [21]:
monthly_product_metrics = (
    suite_NAS_clean
    .groupby(["Product", pd.Grouper(key="InvDate", freq="ME")])
    .agg(Quantity=("Quantity", "sum"), usd_adjusted_total=("usd_adjusted_total", "sum"))
    .reset_index()
    .rename(columns={"usd_adjusted_total": "actual_revenue"})
)


In [22]:
product_prices = (
    suite_NAS_clean.groupby("Product")["usd_adjusted_price"]
    .mean()
    .reset_index()
    .rename(columns={"usd_adjusted_price": "avg_price"})
)

Baseline

In [23]:
# Containers for results
forecasts = []
errors = []

for product, df_prod in monthly_product_metrics.groupby("Product"):
    if len(df_prod) < 15:
        # too few data points, skip
        continue

    df_prod = df_prod.sort_values("InvDate")
    train = df_prod[df_prod["InvDate"] <= "2024-09-30"]
    test  = df_prod[df_prod["InvDate"] >  "2024-09-30"]

    if len(train) < 12:
        continue  # skip if less than one year of data

    # Log transform to stabilize variance
    train_log = np.log(train["Quantity"].replace(0, np.nan)).fillna(method="bfill")
    train_log.name = "log_quantity"

    # Fit SARIMAX
    try:
        model = SARIMAX(
            train_log,
            order=(1,1,1),
            seasonal_order=(1,1,1,12),
            enforce_stationarity=False,
            enforce_invertibility=False,
        )
        fit = model.fit(disp=False)
        forecast = fit.get_forecast(steps=len(test))
        pred_mean = np.exp(forecast.predicted_mean)
        conf_int = forecast.conf_int(alpha=0.05)
        conf_int_exp = np.exp(conf_int)
        lower = conf_int_exp[f"lower {train_log.name}"].values
        upper = conf_int_exp[f"upper {train_log.name}"].values

        # Compute MAPE for this product
        mape = np.mean(np.abs(test["Quantity"].values - pred_mean.values)
                       / np.maximum(test["Quantity"].values, 1)) * 100

        # Save results
        temp = pd.DataFrame({
            "Product": product,
            "forecast_month": pd.date_range(train["InvDate"].max() + pd.offsets.MonthEnd(1),
                                            periods=len(pred_mean),
                                            freq="ME"),
            "forecast_volume": pred_mean.values,
            "forecast_lower": lower,
            "forecast_upper": upper,
            "actual_revenue": test["actual_revenue"].values,
            "actual_volume": test["Quantity"].values
        })
        forecasts.append(temp)
        errors.append({"Product": product, "MAPE": mape})

    except Exception as e:
        print(f"Skipping {product}: {e}")


/var/folders/dt/7vnpb4hn01d19x7dfz27zs5h0000gn/T/ipykernel_85880/1191559154.py:18: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_log = np.log(train["Quantity"].replace(0, np.nan)).fillna(method="bfill")
/Users/tonychu/Git Repository/syno_prediction/.venv/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/var/folders/dt/7vnpb4hn01d19x7dfz27zs5h0000gn/T/ipykernel_85880/1191559154.py:18: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_log = np.log(train["Quantity"].replace(0, np.nan)).fillna(method="bfill")
/Users/tonychu/Git Repository/syno_prediction/.venv/lib/python3.11/site-

In [24]:
# Combine outputs
forecast_df = pd.concat(forecasts, ignore_index=True)
mape_df = pd.DataFrame(errors)

# Align actual revenue from source data in case of missing values
actuals = (
    monthly_product_metrics
    .rename(columns={"InvDate": "forecast_month"})
    [["Product", "forecast_month", "actual_revenue"]]
)

if not forecast_df.empty:
    forecast_df = forecast_df.merge(
        actuals,
        on=["Product", "forecast_month"],
        how="left",
        suffixes=("", "_source"),
    )
    if "actual_revenue_source" in forecast_df.columns:
        forecast_df["actual_revenue"] = (
            forecast_df["actual_revenue"].fillna(forecast_df["actual_revenue_source"])
        )
        forecast_df = forecast_df.drop(columns=["actual_revenue_source"])

# Join prices to forecasts
forecast_df = forecast_df.merge(product_prices, on="Product", how="left")
forecast_df["forecast_revenue"] = forecast_df["forecast_volume"] * forecast_df["avg_price"]

# Round key numeric columns for readability
for col in ["forecast_volume", "forecast_lower", "forecast_upper", "actual_revenue", "avg_price", "forecast_revenue"]:
    if col in forecast_df.columns:
        forecast_df[col] = forecast_df[col].round(2)

# Sort results
forecast_df = forecast_df.sort_values(["Product", "forecast_month"])


In [25]:
forecast_df.head(20)

,Product,forecast_month,forecast_volume,forecast_lower,forecast_upper,actual_revenue,actual_volume,avg_price,forecast_revenue
0,FS2500,2024-10-31,78.21,29.45,207.67,224782.24,52,4177.66,326733.62
1,FS2500,2024-11-30,98.47,36.07,268.83,373454.61,106,4177.66,411382.87
2,FS2500,2024-12-31,75.46,26.92,211.53,310047.39,112,4177.66,315265.61
3,FS3410,2024-10-31,13.83,3.08,62.05,204760.13,38,12071.03,166889.74
4,FS3410,2024-11-30,14.69,2.47,87.30,568773.56,43,12071.03,177323.13
5,FS3410,2024-12-31,18.03,2.22,146.69,441479.30,50,12071.03,217667.84
6,FS3600,2024-10-31,25.92,5.26,127.78,61714.99,8,11343.76,294049.39
7,FS3600,2024-11-30,67.51,0.58,7819.55,268912.21,36,11343.76,765868.16
8,FS3600,2024-12-31,356.68,0.00,46306698.24,333254.05,16,11343.76,4046078.00
9,FS6400,2024-10-31,14.51,5.15,40.85,378226.70,24,19186.65,278320.11


In [26]:
mape_df.head(20)

,Product,MAPE
0,FS2500,30.041994
1,FS3410,64.463169
2,FS3600,813.601187
3,FS6400,25.033438
4,FX2421,47.350867
5,HD6500,35.492161
6,RS1221+,14.124929
7,RS1221RP+,13.056284
8,RS1619xs+,13.086284
9,RS2423+,23.619720


In [27]:
def assign_cohort(mape):
    if mape < 20:
        return "High Confidence"
    elif mape < 50:
        return "Moderate Confidence"
    else:
        return "Low Confidence"


In [28]:
def assign_cohort(mape):
    if mape < 20:
        return "High Confidence"
    elif mape < 50:
        return "Moderate Confidence"
    else:
        return "Low Confidence"

# Step 2: Assign cohort
mape_df["Cohort"] = mape_df["MAPE"].apply(assign_cohort)

# Step 3: Group by cohort and calculate summary stats
cohort_summary = mape_df.groupby("Cohort")["MAPE"].agg(
    Number_of_Products="count",
    Average_MAPE="mean"
).reset_index()

# Step 4: Add tuple of product names
cohort_summary["Products"] = cohort_summary["Cohort"].map(
    mape_df.groupby("Cohort")["Product"].apply(tuple)
)

# Step 5: Show result
print(cohort_summary.to_string(index=False))

             Cohort  Number_of_Products  Average_MAPE                                                                                                                                       Products
    High Confidence                   4     15.062392                                                                                                     (RS1221+, RS1221RP+, RS1619xs+, RS2821RP+)
     Low Confidence                  14    478.522389         (FS3410, FS3600, RS2423RP+, RS4021xs+, RS422+, RS822RP+, RX1217sas, RX1223RP, RX6022sas, RXD1219sas, SA3200D, SA3400D, SA6400, UC3400)
Moderate Confidence                  16     36.046172 (FS2500, FS6400, FX2421, HD6500, RS2423+, RS3618xs, RS3621RPxs, RS3621xs+, RS822+, RX1217, RX1217RP, RX1222sas, RX418, SA3410, SA3610, UC3200)
